In [32]:
import json
import pandas as pd

In [33]:
with open("./query.json", 'r') as f:
    data = json.load(f)['data']['swaps']
for d in data:
    d["transaction_blockNumber"] = d["transaction"]["blockNumber"]
    del d["transaction"]

In [35]:
data[0]["pair"]

{'token0': {'id': '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984',
  'symbol': 'UNI',
  'totalLiquidity': '2363944.259085216407738225',
  'tradeVolume': '764102934.247673078083762688',
  'tradeVolumeUSD': '6641763769.211965729347119956335644'},
 'token1': {'id': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
  'symbol': 'WETH',
  'totalLiquidity': '744520.292490940177891607',
  'tradeVolume': '243285297.752672420784451769',
  'tradeVolumeUSD': '272023920330.3513747372090367518784'},
 'totalSupply': '104813.532696929188491861',
 'volumeUSD': '6632808391.534398018046656895402182'}

In [43]:
fields = [field for field in data[0].keys() if field != "pair"]
token_fields = []
for i in range(2):
    token_fields.extend([f"token{i}_{field}" for field in data[0]["pair"][f"token{i}"] if field not in ["id", "symbol"]])
token_fields.extend([field for field in data[0]["pair"].keys() if "token" not in field])

In [5]:
rawdf = {field: [] for field in fields}
for d in data:
    for field in rawdf.keys():
        if "token" in field:
            token, token_field = field.split('_')
            rawdf[field].append(d["pair"][token][token_field])
        else:
            rawdf[field].append(d[field])
rawdf = pd.DataFrame.from_dict(rawdf)

In [6]:
symbols = [data[0]["pair"][f"token{i}"]["symbol"] for i in range(2)]

rename_cols = {}
for col in rawdf.columns:
    if "amount" in col:
        idx = int(col[col.find("amount") + 6])
        rename_cols[col] = col.replace(f"amount{idx}", f"{symbols[idx]}_")
    if "token" in col:
        idx = int(col[col.find("token") + 5])
        rename_cols[col] = col.replace(f"token{idx}", f"{symbols[idx]}")
rawdf = rawdf.rename(rename_cols, axis=1)

In [15]:
df = rawdf.copy()
for col in rawdf.columns:
    if col != "id":
        df[col] = rawdf[col].astype(float)
    else:
        df[col] = rawdf[col].astype("string")

In [27]:
side = ["Sell" if df.loc[i, "UNI_In"] > 0 else "Buy" for i in range(df.shape[0])]
df["Side"] = pd.Series(side, dtype="string")

: Timestamp, Side (Buy or Sell),
Base currency quantity traded, Quote currency quantity traded, Volume in USD, Liquidity in pool,
Pool ID.

In [30]:
df.columns

Index(['UNI_In', 'UNI_Out', 'WETH_In', 'WETH_Out', 'id', 'timestamp',
       'transaction_blockNumber', 'UNI_totalLiquidity', 'UNI_tradeVolume',
       'UNI_tradeVolumeUSD', 'WETH_totalLiquidity', 'WETH_tradeVolume',
       'WETH_tradeVolumeUSD', 'Side'],
      dtype='object')

In [31]:
cols = {
    "timestamp": "Timestamp",
    "Side": "Side",
    "UNI_tradeVolume": "Base Currency Vol.",
    "WETH_tradeVolume": "Quote Currency Vol.",
    ""
}
col_order = ["timestamp", "Side", "UNI"]